#第6章: 機械学習

本章では，Fabio Gasparetti氏が公開している[News Aggregator Data Set](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)を用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．

##準備

GDriveGDriveのマウント

In [ ]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/NLP

/content
/content/drive/My Drive/Colab Notebooks/NLP


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

--2021-04-09 07:33:05--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  21.6MB/s    in 1.3s    

2021-04-09 07:33:07 (21.6 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]



In [ ]:
!unzip NewsAggregatorDataset.zip

Archive:  NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: 2pageSessions.csv       
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


##課題50. データの入手・整形
News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．

1.   ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
2.   情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
1.   抽出された事例をランダムに並び替える
2.   抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）

学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

###ヒント
pandasのデータフレームで条件を指定して、データの切り取りをする https://note.nkmk.me/python-pandas-query/

pandasで、ある行・列を切り出す　https://note.nkmk.me/python-pandas-at-iat-loc-iloc/

データの分割 https://note.nkmk.me/python-sklearn-train-test-split/

pandasのvalue_counts()で、事例数（相異なる値の数）をとる　https://note.nkmk.me/python-pandas-value-counts/　


###解答例

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    './newsCorpora.csv', 
    header=None, sep='\t', 
    names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
    )

# df.loc[ 行指定, 列指定 ]
df = df.loc[
            df['PUBLISHER'].isin(???????????]),　#<--- 
            ['TITLE', 'CATEGORY']
            ]
print(len(df))
df.to_csv('./news.csv',sep='\t',index=False,header=False)

#https://note.nkmk.me/python-sklearn-train-test-split/
train, valid_test = train_test_split(???,  ???, shuffle=True, stratify=df['CATEGORY'])　　#<---
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, stratify=valid_test['CATEGORY'])

train.to_csv('./train.txt', sep='\t', index=False, header=False)
valid.to_csv('./valid.txt', sep='\t', index=False, header=False)
test.to_csv('./test.txt', sep='\t', index=False, header=False)
!wc -l train.txt valid.txt test.txt

#https://note.nkmk.me/python-pandas-value-counts/
print(train['CATEGORY'].value_counts())
print(valid['CATEGORY'].value_counts())
print(test['CATEGORY'].value_counts())


13340
 10682 train.txt
  1334 valid.txt
  1334 test.txt
 13350 total
b    4502
e    4223
t    1219
m     728
Name: CATEGORY, dtype: int64
b    563
e    528
t    152
m     91
Name: CATEGORY, dtype: int64
b    562
e    528
t    153
m     91
Name: CATEGORY, dtype: int64


##課題51. 特徴量抽出
学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

###ヒント
文書の特徴量 https://qiita.com/fujin/items/b1a7152c2ec2b4963160

教育的な配慮から、単語埋め込みベクトルを用いずに、以下のどちらかを使う

頻度ベクトル　https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

TDIDFベクトル　https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

全単語を見て単語数の次元のベクトルを作るため、train/valid/testに分割する前のデータから、CountVectorizerやTfidfVectorizerを使ってベクトル化してから、再度、train/valid/testに分割したほうがよい。

後続の推論エンジンのトレーニング時に明らかになるが、タイトルに含まれる特殊文字がノイズになって処理がうまくいかない。ベクトル化する前に、データフレーム.str.replace(‘[^a-x]’,’ ‘,regex=True)とか前処理しておく。 https://note.nkmk.me/python-pandas-replace/　

出力は、各行のタイトル（文書）とクラス（正解ラベル）に対応する特徴ベクトルを、*.feature.txtに入れればよい。

###解答例

In [ ]:
#https://qiita.com/fujin/items/b1a7152c2ec2b4963160
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#https://deepage.net/features/pandas-replace.html
#語彙リストを作るのに全データが必要なので、split前の状態から始めるよ
df = pd.read_csv('./news.csv', sep='\t', header=None)
print('入力',df.shape)
#df[0]にはTITLE（記事タイトル文章）が入っている
df[0] = df[0].str.lower()
df[0] = df[0].str.replace('[^a-z]',' ', regex=True)
df[0] = df[0].str.replace(' +',' ', regex=True)

#単語の頻度ベクトルか、TFIDFベクトル
vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
#vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(???????????)　#<---
print('頻度ベクトル',features.shape)

#featuresに頻度ベクトルかTFIDFベクトルが入っているので、それをpandasの配列にする
df_features = pd.DataFrame(features.toarray())
print('Pandas形式の頻度ベクトル',df_features.shape)
data = pd.concat([df,df_features], axis=1)
print('元のTITLE,CATEGORYと頻度ベクトルをつなげたもの',data.shape)

train, valid_test = train_test_split(data,  train_size=0.8, shuffle=True, stratify=data[1])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, stratify=valid_test[1])
print('分割後',train.shape, valid.shape, test.shape)

print('TITLE,CATEGORYデータと、頻度ベクトルを分けて出力')
train.iloc[:, :2].to_csv('train.txt', sep='\t', index=False, header=False)
train.iloc[:,2:].to_csv('train.feature.txt', sep='\t', index=False, header=False)
valid.iloc[:, :2].to_csv('valid.txt', sep='\t', index=False, header=False)
valid.iloc[:,2:].to_csv('valid.feature.txt', sep='\t', index=False, header=False)
test.iloc[:, :2].to_csv('test.txt', sep='\t', index=False, header=False)
test.iloc[:,2:].to_csv('test.feature.txt', sep='\t', index=False, header=False)
!wc -l train.txt train.feature.txt valid.txt valid.feature.txt test.txt test.feature.txt
!head train.txt
!head train.feature.txt

入力 (13340, 2)
頻度ベクトル (13340, 13579)
Pandas形式の頻度ベクトル (13340, 13579)
元のTITLE,CATEGORYと頻度ベクトルをつなげたもの (13340, 13581)
分割後 (10672, 13581) (1334, 13581) (1334, 13581)
TITLE,CATEGORYデータと、頻度ベクトルを分けて出力
    10672 train.txt
    10672 train.feature.txt
     1334 valid.txt
     1334 valid.feature.txt
     1334 test.txt
     1334 test.feature.txt
    26680 total
the other two works are set to go under the hammer at sotheby s day sale 	e
gravitational waves here s everything you need to know video 	t
driverless cars on british roads within a year as ministers change law to allow 	t
us stocks apple lifts nasdaq ukraine drags on broader market	b
glass warfare how google s headgear problems went from bad to worse	t
jd com raises b in initial public offering	b
spare cash in euro zone falls below billion euro threshold	b
emma watson reveals new noah trailer	e
cuba protests record us fine of bnp paribas	b
kanye west kanye west to wed kim kardashian in us ahead of paris 	e
0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

##課題52. 学習
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

###ヒント

参考情報
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

https://qiita.com/0NE_shoT_/items/b702ab482466df6e5569　

52、53は、from sklearn.linear_model import LogisticRegression　を使います。

LogisticRegressionのインスタンスを作成して、そのメソッドfitで、第1引数に特徴ベクトル, 第2引数に正解ラベルを与えてトレーニングする。第一引数のfeature vectorは、tran.feature.txtに入っている。第2引数の分類ラベルは、train.txtの2カラム目に入っている。それらをロードして、LogisticRegressionに渡す。ほかのハイパーパラメータは適当に。


データは、PandasのSeries（件数分だけ縦に並べたもの）で、特徴ベクトルの列、クラスの列を使う。



###解答例

In [ ]:
#https://qiita.com/0NE_shoT_/items/b702ab482466df6e5569
from sklearn.linear_model import LogisticRegression
x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]　#[1]で、2番目のカラム、CATEGORY（正解ラベル）を取り出す
lr = LogisticRegression(random_state=0, max_iter=1000)
lr.fit(?????????, ??????????) #説明変数x_trainx_trainは、頻度ベクトル。目的変数y_trainは正解ラベル #<---

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

##課題53.予測
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

###ヒント


予測は、メソッドpredictに、特徴ベクトルを与えて出力する。 train.feature.txtから読みこんだfeature vectorを引数にして、fitを呼ぶ。

###解答例

In [ ]:
#from sklearn.linear_model import LogisticRegression
#x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
#y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]
#lr = LogisticRegression(random_state=0, max_iter=1000)
#lr.fit(x_train, y_train)
print(lr.predict(????????????)) #<---

['e' 't' 't' ... 't' 'b' 'b']


##課題54. 正解率の計測
52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．

###ヒント

https://note.nkmk.me/python-sklearn-confusion-matrix-score/

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

from sklearn.metrics import accuracy_score を使います。

###解答例

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
#https://note.nkmk.me/python-sklearn-confusion-matrix-score/
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
#y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]
x_test = pd.read_csv('test.feature.txt', sep='\t', header=None)
y_test = pd.read_csv('test.txt', sep='\t', header=None)[1]
#lr = LogisticRegression(random_state=0, max_iter=1000)
#lr.fit(x_train, y_train)
y_train = lr.predict(x_train)
y_test = lr.predict(x_test)
print(accuracy_score(?????????????, ???????????))) #<---
print(accuracy_score(y_test, lr.predict(x_test)))


1.0
1.0


##課題55. 混同行列の作成
52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

###ヒント

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

 from sklearn.metrics import confusion_matrix　を使います。

###解答例

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

'''
x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]
x_test = pd.read_csv('test.feature.txt', sep='\t', header=None)
y_test = pd.read_csv('test.txt', sep='\t', header=None)[1]
lr = LogisticRegression(random_state=0, max_iter=1000)
lr.fit(x_train, y_train)
'''
print('classes:', lr.classes_)
print('train cm\n', confusion_matrix(???????????, ??????????))) #<---
print('test cm\n', confusion_matrix(y_test, lr.predict(x_test)))


classes: ['b' 'e' 'm' 't']
train cm
 [[4510    0    0    0]
 [   0 4224    0    0]
 [   0    0  725    0]
 [   0    0    0 1213]]
test cm
 [[593   0   0   0]
 [  0 560   0   0]
 [  0   0  61   0]
 [  0   0   0 120]]


##課題56. 適合率，再現率，F1スコアの計測
52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．．

###ヒント

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

from sklearn.metrics import classification_report を使います。


###解答例

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

'''x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]
x_test = pd.read_csv('test.feature.txt', sep='\t', header=None)
y_test = pd.read_csv('test.txt', sep='\t', header=None)[1]
lr = LogisticRegression(random_state=0, max_iter=1000)
lr.fit(x_train, y_train)
'''
y_test = lr.predict(x_test)
print(classification_report(??????????, ??????????)))


              precision    recall  f1-score   support

           b       1.00      1.00      1.00       593
           e       1.00      1.00      1.00       560
           m       1.00      1.00      1.00        61
           t       1.00      1.00      1.00       120

    accuracy                           1.00      1334
   macro avg       1.00      1.00      1.00      1334
weighted avg       1.00      1.00      1.00      1334



##課題57. 特徴量の重みの確認　[やらなくてよい]
52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ

[これは、ドキュメントを読み込んで、オブジェクトのデータ構造を深く探らないといけないので、スキップしてください]

###ヒント

参考情報

特徴ベクトルは、オブジェクトのvocabulary_変数に、単語とindexの辞書型を持っている。
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

LogisticRegressionは、オブジェクトのclasses_変数に、分類で使ったラベルのnumpy配列を持っている。また、coef_変数に、学習後の係数を、(クラス数、特徴ベクトル長)のshapeのnumpy配列として持っている。
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

ヒント

クラスごとに、クラスラベルとそれに対応するcoef_行を取り出し、
語彙数の分だけ、vocabulary_情報とcoef_係数が対応して並んでいる。係数の絶対値の大きさでソートして、Tip10、Bottom10を見つけ、それらの単語と係数を出力する。

ソートは、https://note.nkmk.me/python-list-sort-sorted/。

辞書型をsortするには、大小関係を比較するキーを指定す。https://note.nkmk.me/python-dict-list-sort/。

Pythonのlambda式表現で、キー指定する。キーは、lambda式で指定する https://note.nkmk.me/python-lambda-usage/ 。

###解答例

In [ ]:
# 51,52 must be executed 
vocab = vectorizer.vocabulary_
print('vocab:', vocab)
coefs = lr.coef_
print('coefs:', coefs)
cls = lr.classes_
print('classes:', cls)
for c, coef in zip(cls, coefs):
    print('class:', c)
    d = dict(zip(vocab, coef))
    d_top = sorted(d.items(), key=lambda x: abs(x[1]), reverse=True)[:10]
    print('top:', d_top)
    d_bottom = sorted(d.items(), key=lambda x: abs(x[1]), reverse=False)[:10]
    print('bottom:', d_bottom)

vocab: {'europe': 4110, 'reaches': 9649, 'crunch': 2875, 'point': 9017, 'on': 8325, 'banking': 964, 'union': 12710, 'ecb': 3799, 'focus': 4677, 'stronger': 11635, 'euro': 4106, 'drowns': 3692, 'out': 8435, 's': 10341, 'message': 7544, 'to': 12276, 'keep': 6518, 'rates': 9620, 'low': 7119, 'for': 4709, 'anxieties': 539, 'wane': 13096, 'as': 690, 'bunds': 1674, 'top': 12312, 'treasuries': 12446, 'spain': 11254, 'debt': 3072, 'rallies': 9577, 'noyer': 8183, 'says': 10462, 'strong': 11634, 'creates': 2792, 'unwarranted': 12770, 'economic': 3804, 'pressure': 9210, 'refile': 9776, 'bad': 905, 'loan': 7035, 'triggers': 12481, 'key': 6542, 'feature': 4412, 'in': 5967, 'bank': 961, 'test': 12099, 'announcement': 502, 'sources': 11232, 'update': 12778, 'reveal': 10058, 'hurdles': 5827, 'zone': 13573, 'central': 1985, 'banks': 968, 'must': 7936, 'contain': 2586, 'threats': 12190, 'from': 4836, 'prices': 9231, 'threat': 12185, 'not': 8166, 'happy': 5411, 'with': 13360, 'rise': 10156, 'ebay': 3792,

##課題58. 正則化パラメータの変更
ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．

###ヒント
sklearn.linear_mode.LogisticsRegression の任意引数のCを使う。

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

Cが小さいほど、正則化項が強い。


また、Pythonでバッチジョブを回すときに、tqdmがよく用いられる。

https://vector-ium.com/python-tqdm/



###解答例

In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x_train = pd.read_csv('train.feature.txt', sep='\t', header=None)
y_train = pd.read_csv('train.txt', sep='\t', header=None)[1]
x_valid = pd.read_csv('valid.feature.txt', sep='\t', header=None)
y_valid = pd.read_csv('valid.txt', sep='\t', header=None)[1]
x_test = pd.read_csv('test.feature.txt', sep='\t', header=None)
y_test = pd.read_csv('test.txt', sep='\t', header=None)[1]

result = []
for C in tqdm([0.001,0.01,0.1,1,10,100,1000,10000]):
  lr = LogisticRegression(random_state=0, max_iter=1000, C=C)
  lr.fit(x_train, y_train)
  result.append(
      [C, 
       accuracy_score(y_train, lr.predict(x_train)),
       accuracy_score(y_valid, lr.predict(x_valid)),
       accuracy_score(y_test, lr.predict(x_test))
       ])

result = np.array(result).T
plt.plot(result[0], result[1], label='train')
plt.plot(result[0], result[2], label='valid')
plt.plot(result[0], result[3], label='test')
plt.ylabel('Accuracy')
plt.xscale ('log')
plt.xlabel('C')
plt.legend()
plt.show()

 25%|██▌       | 2/8 [00:27<01:27, 14.50s/it]

##課題59. ハイパーパラメータの探索 [スキップしてください]
学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．